## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:



# Project Title

## Overview

## Business Problem

## Data Understanding

We need to find outcomes to solve our business problem:
- tn.movie_budgets.csv.gz contains `production_budget`, `domestic_gross`, `worldwide_gross`
- im.db has `movie_ratings` table with `averagerating` and `numvotes`

We also need features that might relate to our outcomes:
- im.db has movie names, genres, and names of our directors and actors

In [1]:
import pandas as pd
import sqlite3

# print big numbers in pd.DataFrame (instead of scientific notation)
pd.set_option('display.precision', 12)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Getting financial info from tn.movie_budgets.csv.gz
- Contains `production_budget`, `domestic_gross`, `worldwide_gross` formatted as string with special characters (e.g., $425,000,000)

In [2]:
tn_movie_budgets_df = pd.read_csv('../zippedData/tn.movie_budgets.csv.gz')

print(tn_movie_budgets_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB
None


In [3]:
print(tn_movie_budgets_df.head())
print(tn_movie_budgets_df.tail())

   id  release_date                                        movie  \
0   1  Dec 18, 2009                                       Avatar   
1   2  May 20, 2011  Pirates of the Caribbean: On Stranger Tides   
2   3   Jun 7, 2019                                 Dark Phoenix   
3   4   May 1, 2015                      Avengers: Age of Ultron   
4   5  Dec 15, 2017            Star Wars Ep. VIII: The Last Jedi   

  production_budget domestic_gross worldwide_gross  
0      $425,000,000   $760,507,625  $2,776,345,279  
1      $410,600,000   $241,063,875  $1,045,663,875  
2      $350,000,000    $42,762,350    $149,762,350  
3      $330,600,000   $459,005,868  $1,403,013,963  
4      $317,000,000   $620,181,382  $1,316,721,747  
      id  release_date                          movie production_budget  \
5777  78  Dec 31, 2018                         Red 11            $7,000   
5778  79   Apr 2, 1999                      Following            $6,000   
5779  80  Jul 13, 2005  Return to the Land of Wo

### Getting information about tables and columns in im.db


In [4]:
# Create the connect to im.db database
con = sqlite3.connect('../zippedData/im.db')

In [5]:
# Get info about tables and columns
im_schema_df = pd.read_sql("""

SELECT *
FROM sqlite_master

""", con)

In [6]:
im_schema_df

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [7]:
# Let's see column names and types in sql for each table
for idx in im_schema_df.index:
    print(im_schema_df.iloc[idx]['name'], im_schema_df.iloc[idx]['sql'])

movie_basics CREATE TABLE "movie_basics" (
"movie_id" TEXT,
  "primary_title" TEXT,
  "original_title" TEXT,
  "start_year" INTEGER,
  "runtime_minutes" REAL,
  "genres" TEXT
)
directors CREATE TABLE "directors" (
"movie_id" TEXT,
  "person_id" TEXT
)
known_for CREATE TABLE "known_for" (
"person_id" TEXT,
  "movie_id" TEXT
)
movie_akas CREATE TABLE "movie_akas" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "title" TEXT,
  "region" TEXT,
  "language" TEXT,
  "types" TEXT,
  "attributes" TEXT,
  "is_original_title" REAL
)
movie_ratings CREATE TABLE "movie_ratings" (
"movie_id" TEXT,
  "averagerating" REAL,
  "numvotes" INTEGER
)
persons CREATE TABLE "persons" (
"person_id" TEXT,
  "primary_name" TEXT,
  "birth_year" REAL,
  "death_year" REAL,
  "primary_profession" TEXT
)
principals CREATE TABLE "principals" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "person_id" TEXT,
  "category" TEXT,
  "job" TEXT,
  "characters" TEXT
)
writers CREATE TABLE "writers" (
"movie_id" TEXT,
  "person_id" TEXT


#### Use SQL to get info about `movie_basics` and `movie_ratings`
- `movie_ratings`
    - `averagerating` and `numvotes` are columns of imdb ratings
- `movie_basics` table
    - `genres` is a string that contains multiple genres (separated by commas)
    - `primary_title` has duplicate rows with same value

In [8]:
# What do sample values look like?
pd.read_sql("""
SELECT
    *
FROM
    movie_ratings
LIMIT 5
""", con)

,movie_id,averagerating,numvotes
0,tt10356526,8.30,31
1,tt10384606,8.90,559
2,tt1042974,6.40,20
3,tt1043726,4.20,50352
4,tt1060240,6.50,21


In [9]:
# How many records exist? What are the MIN() and MAX() values
pd.read_sql("""
SELECT
    COUNT(*) AS movie_ratings_rows,
    MIN(averagerating) AS min_averagerating,
    MAX(averagerating) AS max_averagerating,
    MIN(numvotes) AS min_numvotes,
    MAX(numvotes) AS max_numvotes
FROM
    movie_ratings
""", con)

,movie_ratings_rows,min_averagerating,max_averagerating,min_numvotes,max_numvotes
0,73856,1.00,10.00,5,1841066


In [10]:
# What do sample values look like?
pd.read_sql("""
SELECT
    *
FROM
    movie_basics
LIMIT 5
""", con)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.00,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.00,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.00,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,nan,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.00,"Comedy,Drama,Fantasy"


In [11]:
# How many records exist? What are the MIN() and MAX() values
pd.read_sql("""
SELECT
    COUNT(*) as movie_basics_rows,
    MIN(runtime_minutes) AS min_runtime_minutes,
    MAX(runtime_minutes) AS max_runtime_minutes,
    MIN(start_year) AS min_start_year,
    MAX(start_year) AS max_start_year
FROM
    movie_basics
""", con)

,movie_basics_rows,min_runtime_minutes,max_runtime_minutes,min_start_year,max_start_year
0,146144,1.00,51420.00,2010,2115


#### Use SQL to get info about  `directors`, `persons`, `principals`
 

In [12]:
# What do sample values look like?
pd.read_sql("""
SELECT
    *
FROM
    directors
LIMIT 5
""", con)

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


In [13]:
# How many records exist?
pd.read_sql("""
SELECT
    COUNT(*) as directors_rows
FROM
    directors
""", con)

,directors_rows
0,291174


In [14]:
# What do sample values look like?
pd.read_sql("""
SELECT
    *
FROM
    known_for
LIMIT 5
""", con)

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534


In [15]:
# How many records exist?
pd.read_sql("""
SELECT
    COUNT(*) as known_for_rows
FROM
    known_for
""", con)

,known_for_rows
0,1638260


In [16]:
# What do sample values look like?
pd.read_sql("""
SELECT
    *
FROM
    persons
LIMIT 5
""", con)

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,None,None,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,None,None,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,None,None,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,None,None,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,None,None,"production_designer,art_department,set_decorator"


In [17]:
# How many records exist?
pd.read_sql("""
SELECT
    COUNT(*) as persons_rows
FROM
    known_for
""", con)

,persons_rows
0,1638260


In [18]:
# What do sample values look like?
pd.read_sql("""
SELECT
    *
FROM
    principals
LIMIT 5
""", con)

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"


In [27]:
# How many records exist?
pd.read_sql("""
SELECT
    COUNT(*) as principals_rows
FROM
    principals
""", con)

,principals_rows
0,1028186


In [20]:

pd.read_sql("""
WITH distinct_rows AS (
    SELECT
        DISTINCT movie_id, person_id
    FROM
        principals
)
SELECT
    COUNT(*) as principals_distinct_rows
FROM
    distinct_rows

""", con)

,principals_distinct_rows
0,1028148


## Data Preparation

### Extract data from sqlite3 im.db into convenient pd.DataFrame(s)

#### Let's try to get data from im.db `movie_basics`  and `movie_ratings`

In [21]:
### Look into who the primary directors are
imdb_ratings_df = pd.read_sql("""

SELECT
    b.movie_id, b.primary_title, b.original_title, b.start_year, b.runtime_minutes, b.genres,
    r.averagerating, r.numvotes
FROM
    -- Use LEFT JOIN to keep all movies, even if they have no ratings
    movie_basics as b
    LEFT JOIN movie_ratings as r
        USING(movie_id)

""", con)
imdb_ratings_df

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.00,"Action,Crime,Drama",7.00,77.00
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.00,"Biography,Drama",7.20,43.00
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.00,Drama,6.90,4517.00
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,nan,"Comedy,Drama",6.10,13.00
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.00,"Comedy,Drama,Fantasy",6.50,119.00
...,...,...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.00,Drama,nan,nan
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,nan,Documentary,nan,nan
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,nan,Comedy,nan,nan
146142,tt9916730,6 Gunn,6 Gunn,2017,116.00,None,nan,nan


#### Descriptives on imdb_ratings_df
- in part, ran this to check that 
    - movie_basics returned 146144 rows, and 
    - movie_ratings returned 73856 rows
- max values also show some weirdness in `start_year`, `runtime_minutes`, and possibly `numvotes`


In [22]:
imdb_ratings_df.describe()

,start_year,runtime_minutes,averagerating,numvotes
count,146144.00,114405.00,73856.00,73856.00
mean,2014.62,86.19,6.33,3523.66
std,2.73,166.36,1.47,30294.02
min,2010.00,1.00,1.00,5.00
25%,2012.00,70.00,5.50,14.00
50%,2015.00,87.00,6.50,49.00
75%,2017.00,99.00,7.40,282.00
max,2115.00,51420.00,10.00,1841066.00


#### Let's try to get data from im.db `persons`  and `principals`
- I'm keeping outputs of this JOIN separate, because relationship of `movie_id` to `person_id` is one-to-many (i.e., a `person_id` can have multiple roles in one `movie_id`)
- There are multiple directors for some movies, and we will need to figure out how to deal with that when determining which directors are best

In [23]:
imdb_principals_df = pd.read_sql("""

SELECT
    *
FROM
    -- Use LEFT JOIN to keep all movies, even if they have no ratings
    principals
    LEFT JOIN persons
        USING(person_id)

""", con)
imdb_principals_df

,movie_id,ordering,person_id,category,job,characters,primary_name,birth_year,death_year,primary_profession
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]",Tommy Dysart,nan,nan,actor
1,tt0111414,2,nm0398271,director,None,None,Frank Howson,1952.00,nan,"actor,writer,producer"
2,tt0111414,3,nm3739909,producer,producer,None,Barry Porter-Robinson,nan,nan,"producer,art_department"
3,tt0323808,10,nm0059247,editor,None,None,Sean Barton,1944.00,nan,"editor,editorial_department,assistant_director"
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]",Brittania Nicol,nan,nan,"actress,soundtrack"
...,...,...,...,...,...,...,...,...,...,...
1028181,tt9692684,1,nm0186469,actor,None,"[""Ebenezer Scrooge""]",Kenneth Cranham,1944.00,nan,"actor,soundtrack"
1028182,tt9692684,2,nm4929530,self,None,"[""Herself"",""Regan""]",Pearl Mackie,1987.00,nan,actress
1028183,tt9692684,3,nm10441594,director,None,None,Guy Jones,nan,nan,director
1028184,tt9692684,4,nm6009913,writer,writer,None,Sabrina Mahfouz,nan,nan,writer


### Data Cleaning

#### Let's make some helper functions
- Something to convert $123,456,789.00 to float


In [24]:
def convert_money_string(money_series):
    '''
    This function takes a pd.Series with string values representing money (USD) and converts that string to float
    '''
    return money_series.str.replace('$','').str.replace(',','').astype(float)

In [25]:
filt = imdb_ratings_df['primary_title']=='Avatar' # Not the real Avatar
imdb_ratings_df[filt]

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
17060,tt1775309,Avatar,Abatâ,2011,93.00,Horror,6.10,43.00


In [28]:
filt = (imdb_principals_df['primary_name'] == 'James Cameron')
imdb_principals_df[filt]

,movie_id,ordering,person_id,category,job,characters,primary_name,birth_year,death_year,primary_profession
41036,tt2004304,2,nm0000116,self,None,"[""Himself""]",James Cameron,1954.00,nan,"writer,producer,director"
41690,tt2169432,1,nm0000116,self,None,"[""Himself""]",James Cameron,1954.00,nan,"writer,producer,director"
81675,tt1927124,1,nm0000116,self,None,"[""Himself""]",James Cameron,1954.00,nan,"writer,producer,director"
106933,tt0437086,6,nm0000116,writer,screenplay by,None,James Cameron,1954.00,nan,"writer,producer,director"
121832,tt1340138,8,nm0000116,writer,based on characters created by,None,James Cameron,1954.00,nan,"writer,producer,director"
143875,tt2306723,2,nm0000116,self,None,"[""Himself""]",James Cameron,1954.00,nan,"writer,producer,director"
146011,tt1745826,1,nm0000116,self,None,"[""Himself""]",James Cameron,1954.00,nan,"writer,producer,director"
204057,tt2332883,1,nm0000116,self,None,"[""Himself""]",James Cameron,1954.00,nan,"writer,producer,director"
221423,tt2013207,2,nm0000116,self,None,"[""Himself""]",James Cameron,1954.00,nan,"writer,producer,director"
234860,tt1757678,5,nm0000116,director,None,None,James Cameron,1954.00,nan,"writer,producer,director"


In [32]:
filt = (imdb_principals_df['primary_name'] == 'James Cameron') & ((imdb_principals_df['category'] == 'director'))
filt = imdb_ratings_df['movie_id'].isin(list(imdb_principals_df[filt].movie_id))
imdb_ratings_df[filt]

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
11392,tt1630029,Avatar 2,Avatar 2,2021,nan,"Action,Adventure,Fantasy",nan,nan
16337,tt1757678,Avatar 3,Avatar 3,2023,nan,"Action,Adventure,Drama",nan,nan
52213,tt3095356,Avatar 4,Avatar 4,2025,nan,"Action,Adventure,Fantasy",nan,nan
96592,tt5637536,Avatar 5,Avatar 5,2027,nan,"Action,Adventure,Fantasy",nan,nan


### Merging Datasets

#### Use SQL to JOIN `movie_basics` and `movie_ratings`
- `movie_basics` table
    - `genres` is a string that contains multiple genres (separated by commas)
    - `primary_title` has duplicate rows with same value
- `movie_ratings`
    - `averagerating` and `numvotes` are columns of imdb ratings

### Feature Engineering
- Create `net_profit` and `roi` columns

## Analysis

### Recommendation 1


### Recommendation 1


### Recommendation 1


## Conclusions

## Next Steps